# Import packages and libraries
## Import the packages and libraries to use

In [1]:
import os
import numpy as np
from PIL import Image
from numpy import asarray
import tarfile

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten 
from keras.layers import Dense 
from keras.layers import Dropout
from keras.optimizers import SGD, schedules
from keras.preprocessing.image import ImageDataGenerator
from keras import initializers

# Extract the Dataset
## Input the zip file from object storage and extract the data

In [ ]:
# Extracting D1 train
# my_tar = tarfile.open('./drive/My Drive/Plant Classification/Dataset/D1/train.tar')
# my_tar.extractall('./drive/My Drive/Plant Classification/Dataset/D1')
# my_tar.close()

In [ ]:
# Extracting D1 test
# my_tar = tarfile.open('./drive/My Drive/Plant Classification/Dataset/D1/test.tar')
# my_tar.extractall('./drive/My Drive/Plant Classification/Dataset/D1')
# my_tar.close()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create the Datset

In [2]:
#D1 Train

root_directory = "./MK/D1/train"
trainX = []
trainY = []

#getting the labels and converting the image into array from the training image
for classNo in range(0,44):
  directory = root_directory + "/Class (" + str(classNo+1) + ")"
  for image_name in os.listdir(directory):
    image_path = os.path.join(directory, image_name)
    image = Image.open(image_path)
    img1 = image.crop((14,14, image.width-14, image.height-14))
    image_array = asarray(img1)
    trainX.append(image_array)
    one_hot = np.zeros((44))
    one_hot[classNo] = 1
    trainY.append(one_hot)

trainX = np.array(trainX)
trainY = np.array(trainY)
print("trainX: ", trainX.shape)
print("trainY: ", trainY.shape)

trainX:  (2288, 228, 228, 3)
trainY:  (2288, 44)


In [3]:
#D1 Test 

root_directory = "./MK/D1/test"
testX = []
testY = []

#getting the labels and converting the image into array from the test image
for classNo in range(0,44):
  directory = root_directory + "/Class (" + str(classNo+1) + ")"
  for image_name in os.listdir(directory):
    image_path = os.path.join(directory, image_name)
    image = Image.open(image_path)
    img1 = image.crop((14,14, image.width-14, image.height-14))
    image_array = asarray(img1)
    testX.append(image_array)
    one_hot = np.zeros((44))
    one_hot[classNo] = 1
    testY.append(one_hot)

testX = np.array(testX)
testY = np.array(testY)
print("testX: ", testX.shape)
print("testY: ", testY.shape)

testX:  (528, 228, 228, 3)
testY:  (528, 44)


In [4]:
#Image crop trial
img_file_path = "./MK/D1/test/Class (1)/R_0Class1 (12).jpg"
img = Image.open(img_file_path)
img_array = asarray(image)
print(image_array.shape)
img1 = img.crop((14,14, img.width-14, img.height-14))
img_array = asarray(img1)
print(img_array.shape)


(228, 228, 3)
(228, 228, 3)


#  Build the Model

In [5]:
# CNN for D1

classifier=Sequential()
classifier.add(Conv2D(filters=96, kernel_size=11, strides=4, input_shape=(228, 228, 3), activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01)))
classifier.add(MaxPooling2D(pool_size=(3,3), strides=2))
classifier.add(Conv2D(filters=256, kernel_size=5, strides=1, padding="same", activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01)))
classifier.add(MaxPooling2D(pool_size=(3,3), strides=2))
classifier.add(Conv2D(filters=384, kernel_size=3, strides=1, padding="same", activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01)))
classifier.add(Conv2D(filters=384, kernel_size=3, strides=1, padding="same", activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01)))
classifier.add(Conv2D(filters=256, kernel_size=3, strides=1, padding="same", activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01)))
classifier.add(MaxPooling2D(pool_size=(3,3), strides=2))
classifier.add(Flatten())
classifier.add(Dense(units=4096, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.005)))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=4096, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.005)))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=44, activation='softmax', kernel_initializer=initializers.RandomNormal(stddev=0.01)))
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=20000,
    decay_rate=0.1)
classifier.compile(optimizer=SGD(learning_rate=lr_schedule, momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

# Train the Model

In [9]:
classifier.fit(
    x=trainX,
    y=trainY,
    batch_size=50,
    epochs=100,
    validation_data=(testX, testY)
)

Epoch 1/100
46/46 [==============================] - 219s 5s/step - loss: 3.7844 - accuracy: 0.0210 - val_loss: 3.7814 - val_accuracy: 0.0265
Epoch 2/100
46/46 [==============================] - 163s 4s/step - loss: 3.7806 - accuracy: 0.0236 - val_loss: 3.7776 - val_accuracy: 0.0227
Epoch 3/100
46/46 [==============================] - 160s 3s/step - loss: 3.7788 - accuracy: 0.0275 - val_loss: 3.7716 - val_accuracy: 0.0246
Epoch 4/100
46/46 [==============================] - 157s 3s/step - loss: 3.7735 - accuracy: 0.0275 - val_loss: 3.7660 - val_accuracy: 0.0227
Epoch 5/100
46/46 [==============================] - 164s 4s/step - loss: 3.7707 - accuracy: 0.0240 - val_loss: 3.7588 - val_accuracy: 0.0322
Epoch 6/100
46/46 [==============================] - 164s 4s/step - loss: 3.7546 - accuracy: 0.0306 - val_loss: 3.7271 - val_accuracy: 0.0549
Epoch 7/100
46/46 [==============================] - 163s 4s/step - loss: 3.7145 - accuracy: 0.0341 - val_loss: 3.6494 - val_accuracy: 0.0473
Epoch 

46/46 [==============================] - 170s 4s/step - loss: 0.0857 - accuracy: 0.9720 - val_loss: 1.6264 - val_accuracy: 0.6402
Epoch 59/100
46/46 [==============================] - 163s 4s/step - loss: 0.1070 - accuracy: 0.9698 - val_loss: 1.6209 - val_accuracy: 0.6269
Epoch 60/100
46/46 [==============================] - 172s 4s/step - loss: 0.1312 - accuracy: 0.9572 - val_loss: 1.6359 - val_accuracy: 0.6288
Epoch 61/100
46/46 [==============================] - 170s 4s/step - loss: 0.0750 - accuracy: 0.9768 - val_loss: 1.4704 - val_accuracy: 0.6458
Epoch 62/100
46/46 [==============================] - 165s 4s/step - loss: 0.0631 - accuracy: 0.9843 - val_loss: 1.6185 - val_accuracy: 0.6553
Epoch 63/100
46/46 [==============================] - 170s 4s/step - loss: 0.0479 - accuracy: 0.9825 - val_loss: 2.0147 - val_accuracy: 0.5833
Epoch 64/100
46/46 [==============================] - 166s 4s/step - loss: 0.0529 - accuracy: 0.9830 - val_loss: 1.8539 - val_accuracy: 0.6402
Epoch 65/100